Импортируем нужные для работы библиотеки

In [1]:
! pip install spacy
! python -m spacy download ru_core_news_sm

                                              0.0/15.3 MB ? eta -:--:--
                                              0.0/15.3 MB 1.3 MB/s eta 0:00:12
                                              0.1/15.3 MB 1.2 MB/s eta 0:00:14
                                              0.1/15.3 MB 1.1 MB/s eta 0:00:14
                                              0.2/15.3 MB 1.1 MB/s eta 0:00:14
                                              0.2/15.3 MB 1.3 MB/s eta 0:00:12
                                              0.3/15.3 MB 1.4 MB/s eta 0:00:11
     -                                        0.4/15.3 MB 1.5 MB/s eta 0:00:10
     -                                        0.5/15.3 MB 1.6 MB/s eta 0:00:10
     -                                        0.6/15.3 MB 1.7 MB/s eta 0:00:09
     -                                        0.7/15.3 MB 1.7 MB/s eta 0:00:09
     --                                       0.8/15.3 MB 1.8 MB/s eta 0:00:08
     --                                       0.9/15.3 MB 1

In [2]:
import pandas as pd
import unicodedata
import re
import spacy
import json
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import json

# Предобработка данных

Загрузим данные

In [3]:
data = pd.read_csv('csv\\full_df.csv', sep=';')
data.head(3)

,Unnamed: 0,status,url,text,likes,reposts,comments
0,0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0
1,1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0
2,2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11238 entries, 0 to 11237
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  11238 non-null  int64  
 1   status      11238 non-null  object 
 2   url         11238 non-null  object 
 3   text        11238 non-null  object 
 4   likes       11081 non-null  object 
 5   reposts     10908 non-null  float64
 6   comments    10902 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 614.7+ KB


In [5]:
data = data.drop(columns='Unnamed: 0', axis=1)

Отфильтруем данные по количеству символов в постах. Оставим посты только длиннее 100 символов.

In [6]:
data['text_len'] = data.loc[:, 'text'].apply(lambda x: len(x))
data_filtered_by_text_len = data.query('text_len > 100')
print(f'Количество постов с 100+ символами: {data_filtered_by_text_len.shape[0]}')
data_filtered_by_text_len.head(3)

Количество постов с 100+ символами: 8556


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Для обработки нам нужны только посты на кириллице. Отфильтруем посты содержащие кириллицу.

In [7]:
data_cyrillic = data_filtered_by_text_len[data_filtered_by_text_len['text'].apply(lambda x: re.match(r'[А-Яа-я]+', x) is not None)]

print(f'Текстов на кириллице: {data_cyrillic.shape[0]}')
data_cyrillic.head(3)

Текстов на кириллице: 1960


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
2,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",17.0,6.0,0.0,117


Удалим дубликаты в столбце с текстами

In [8]:
data_dd = data_cyrillic.drop_duplicates('text')
print(f'Осталось {data_dd.shape[0]} строк')
data_dd.head(3)

Осталось 1862 строк


,status,url,text,likes,reposts,comments,text_len
0,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,Всем привет!Выпустил свою первую статью на хаб...,17.0,0.0,0.0,295
1,Middle Software Engineer - Yandex,https://www.linkedin.com/in/michilegorov,"Если вам интересно позалипать в слова, я запус...",6.0,0.0,2.0,117
6,Talent Acquisition Manager | Recruitment Lead ...,https://www.linkedin.com/in/dariaivanova,С трепетом готов представить свой продукт. Реф...,8.0,0.0,0.0,841


# Подготовка текстов

In [9]:
model = spacy.load('ru_core_news_sm', disable=['ner', 'parser'])

Складываем все посты в один список

In [10]:
texts = data_dd["text"].tolist()

Выполняем лемматизацию текстов. Наберитесь терпения - это долгий процесс, который может занять около часа.

In [11]:
if os.path.exists('lemmas.json'):
    with open("lemmas.json") as f:
        res = json.load(f)['lemmas']
else:
    res = []
    for doc in model.pipe(texts, disable=["tagger", "parser"]):
        res.append([token.lemma_ for token in doc])

    with open("lemmas.json", "w") as fid:
        json.dump({"lemmas": res}, fid)

Смотрим на результат - случайным образом сэмплируем несколько примеров

In [12]:
", ".join(res[np.random.randint(len(res))])

'зачем, же, мне, linkedin, ,, если, не, мочь, помочь, найти, классный, место, работа, для, свой, друг, ,, а, ещё, и, по, совместительство, крутой, специалист, 😉, за, последний, год, тамир, набраться, коммерческий, опыт, ,, пройти, или, проходить, необходимый, ему, курс, ,, и, готовый, приступать, к, работа, в, ваш, компания, в, качество, product, manager, /, OwnerМного, хвалить, не, быть, ,, а, то, ещё, сглажуи, в, что, я, точно, уверенный, ,, что, равнодушный, он, никто, не, оставить, 👍, dm, tamir, sadvakassov, or, tg, @sadvakassov'

In [13]:
", ".join(res[np.random.randint(len(res))])

'тренд, в, мобильный, разработкесовременный, мир, невозможный, представить, без, мобильный, устройство, и, приложение, ., несмотря, на, уже, достигнуть, высота, ,, мобильный, разработка, продолжать, активно, развиваться, ,, поэтому, существовать, определённый, тенденция, ,, которые, наиболее, ярко, проявляться, в, этот, сфера, ., развитие, технологийсовременные, мобильный, приложение, становиться, все, более, сложный, и, функциональный, ,, что, требовать, развитие, технология, ., одной, из, главный, тенденция, являться, развитие, и, усовершенствование, cross, -, platform, -, технология, ,, такой, как, react, native, ,, xamarin, и, flutter, ,, которые, позволять, создавать, мобильный, приложение, для, несколько, платформа, одновременно, ., разработка, без, кодаодной, из, новый, тенденция, в, мобильный, разработка, являться, разработка, без, код, ., это, подход, ,, который, позволять, создавать, приложение, без, необходимость, писать, код, ., вместо, это, ,, разработчик, использовать, гр

Удалим знаки препинания, цифры и слова, содержащие цифры. Для это соберём регулярное выражение, которое подходит для любого слова, состоящего из букв кирилицы.

In [14]:
word_pattern = re.compile("^[а-я]*$")

def remove_symbols(doc):
    return [token for token in doc if word_pattern.match(token)]

corpus = list(map(remove_symbols, res))

In [15]:
", ".join(corpus[np.random.randint(len(corpus))])

'мочь, ли, общение, с, управлять, компания, быть, удобный, комфортный, и, оперативный, наш, заказчик, управлять, компания, тсн, которая, администрировать, несколько, десяток, коттеджный, решить, создать, удобный, веб, сервис, и, мобильный, приложение, в, соответствие, с, концепция, жилец, мочь, контролировать, расход, и, настраивать, автоплатежи, просматривать, юридический, документ, и, поддерживать, связь, с, ук, узнавать, новость, и, план, по, благоустройство, принимать, участие, в, опрос, и, голосованиях, онлайн, управлять, аккаунт, с, ноутбук, и, мобильный, телефон, и, многое, другое, более, подробно, узнать, о, проект, можно, в, кейсе'

# Векторизация текстов

Теперь необходимо перевести тексты новостей и слова, из которых они состоят, в числовое представление, т.е. провести векторизацию. Для этого можно использовать подход "мешок слов" (Bag of Words) или Tf-iDf. Второй вариант нравится мне больше.

Чтобы уменьшить размерность получаемой матрицы, добавим исключение стоп-слов.

In [16]:
stop_words = stopwords.words('russian')
print(stop_words)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [17]:
tfidf = TfidfVectorizer(stop_words=stop_words, min_df=10, max_df=0.9)

Т.е. TfIdfVectorizer принимает на вход список предложений, поэтому необходимо объединить списки слов, составляющих корпус, в предложения.

In [18]:
docs = [" ".join(tokens) for tokens in corpus]
docs[:3]

['всем свой первый статья на про история и технология разработка игра и как мы создать игра с элемент машинный обучение и выйти в ноль за месяцапри внимательный прочтение вы даже смочь запустить первый версия игра',
 'если вам интересный позалипать в слово я запустить понравиться брат шникам',
 'с трепет готовый представить свой продукт рефни первый сервис реферального рекрутинг в ит на нем каждый мочь посоветовать знакомого на открытый позиция в компания и получить за рекомендация денежный вознаграждение идея создание внешний реферального сервис зародиться у меня давно в период пандемия я посоветовать пара человек на хороший должность в разный компания их взять все счастливый тогда я и подумать пора создание команда подготовка тз изучение самых разный нюанс о которых я ранний не знать и самое волнительный поиск инвестор все получиться и весь это теперь позади а впереди только постоянный развитие продукт посмотреть что у нас выйти быть благодарный если вы поддержать меня и мой команда 

In [19]:
x = tfidf.fit_transform(docs)

In [20]:
x.shape

(1862, 1635)

Теперь составим словарь {id_токена: токен} - он пригодится нам позднее.

In [21]:
id2word = {i: token for i, token in enumerate(tfidf.get_feature_names_out())}

# LDA

Теперь можем запустить алгоритм LDA. Зададим количество тем равное 20.

In [22]:
N_TOPICS = 20

In [23]:
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=765)

In [24]:
topics = lda.fit_transform(x)

In [25]:
topics.shape

(1862, 20)

Получили массив размера 1862x20. Строки здесь - отдельные тексты, а колонки - 20 выделенных тем. В каждой ячейке стоит вероятность того, что данный текст относится к данной теме. Это то самое распределение тем над документами.

## Ключевые слова

Теперь извлечём ключевые слова для каждой из 20 тем. В свойсте components _ лежит матрица соответствия слов темам. Пройдёмся по каждой из 20 выделенных тем и для каждой возьмём ТОП-20 слов.

По очереди пройдёмся по каждой теме (первое измерение массиа) и выберем ТОП-20 слов, наиболее характерных для этой темы. Потом по индексу слова обратимся к словарю и узнаем, что это за слово.

In [26]:
for i in range(N_TOPICS):
    c = lda.components_[i, :]
    topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-20:]]
    print("Тема",i+1, ":")
    print(", ".join(topic_tokens))
    print("\n ")

Тема 1 :
сторона, готовый, присылать, яка, место, знать, сюда, нагрузка, снижение, стоить, сегодня, спросить, дать, талант, помнить, кофе, недвижимость, картинка, результат, працювати

 
Тема 2 :
интересный, которые, разработка, нужный, первый, команда, работать, очень, проект, время, новый, компания, мочь, работа, наш, год, человек, весь, свой, это

 
Тема 3 :
ит, компания, пакет, внутренний, знание, ситуация, уровень, данные, который, заказчик, требование, база, управление, опыт, группа, анализ, рост, аналитик, работа, данных

 
Тема 4 :
информация, доступ, захотеться, коммерческий, представить, писать, некоторый, интеграция, библиотека, начать, тема, текстовый, здравствовать, модуль, формировать, маркетинг, изображение, поисковый, интересовать, отслеживать

 
Тема 5 :
проектный, разработка, нам, форум, приглашать, наш, студия, приглашение, конференция, игровой, коллектив, приводить, изображение, коммент, сегодня, нейросети, команда, добрый, машинноеобучение, компьютерноезрение

 
Те

По полученным ключевым словам можно составить представление о содержании той или иной темы. Хотя иногда сочетания слов довольно странные )

## Типичные статьи

Теперь для каждой темы возьмём по одному документу, который хорошо соответсвует этой теме. Для этого заглянем в массив topics.

In [27]:
for i in range(N_TOPICS):
    doc_id = np.argmax(topics[:, i])
    print("Тема ", i+1)
    print(data_dd.iloc[doc_id]["text"])
    print("\n")

Тема  0
Недвижимость в Европе?Уникальные готовые к заселению резиденции расположились в роскошном средиземноморском городке Portonovi в самом сердце Бока-Которского залива.🔹 Цены от €530.000 Присылайте заявку для связи с консультантом по недвижимости.


Тема  1
Всем привет 👋 Я уже больше двух лет увлекаюсь web-разработкой и столько же думаю о том, чтобы сделать её основной профессией. Ещё недавно я работал инженером-конструктором и разрабатывал подстанции. Эта работа мне нравилась и я хорошо себя на ней зарекомендовал. Несмотря на это я оказался в списке тех, кого нужно было уволить к новому году, сегодня это уже не удивляет 😅За последние годы я успел поработать на фрилансе как верстальщик и реализовать несколько личных pet-проектов в основном на VueJS и expressJS.Я понимаю, что мне многому нужно ещё научиться. Однако также понимаю, что огромный путь боли я прошел сам и знаю, что такое учиться программировать 😱 Сейчас очень хочу попасть в команду единомышленников, в которой я смогу реа

# Тест

In [32]:
def lda(n_topics):
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=765)
    topics = lda.fit_transform(x)
    print(topics.shape)

    for i in range(n_topics):
        c = lda.components_[i, :]
        topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-20:]]
        print("Тема",i+1, ":")
        print(", ".join(topic_tokens))
        print("\n ")

    for i in range(n_topics):
        doc_id = np.argmax(topics[:, i])
        print("Тема ", i+1)
        print(data_dd.iloc[doc_id]["text"])
        print("\n")

In [33]:
lda(10)

(1862, 10)
Тема 1 :
задача, сделать, новый, хотеть, курс, делать, работать, первый, которые, нужный, вопрос, очень, год, время, весь, работа, человек, мочь, свой, это

 
Тема 2 :
буде, ваш, може, чи, свой, ви, або, якщо, проект, можна, але, роботи, також, день, ми, це, час, наш, як, що

 
Тема 3 :
приложение, возможность, всем, привет, знание, специалист, продукт, новый, бизнес, год, искать, вакансия, разработчик, наш, компания, разработка, проект, опыт, работа, команда

 
Тема 4 :
новый, поддержка, карьерный, совет, предложить, поиск, рекомендация, контакт, ваш, заранее, здравствовать, благодарить, хотеть, возможность, пройти, следующий, проверка, значок, тест, навык

 
Тема 5 :
рассказать, коллега, декабрь, бот, движение, вебинар, технология, приглашать, достижение, зп, кейс, мир, рассказывать, форум, обеспечение, кибербезопасность, инфобез, сегодня, иб, добрый

 
Тема 6 :
заходить, приводить, сайт, программа, яндекс, стажировка, стартовать, обновить, управлениепроектами, погнать, ко

In [34]:
lda(40)

(1862, 40)
Тема 1 :
онлайн, продукция, статья, версия, контент, качественный, обеспечивать, решение, понравиться, задача, главный, устроить, отрасль, инженер, личность, методика, самому, разрабатывать, самостоятельный, кнопка

 
Тема 2 :
також, день, применение, цього, буде, розвитку, бути, коли, працювати, чи, який, це, час, що, обзор, ви, як, инфобез, кибербезопасность, иб

 
Тема 3 :
описание, стресс, связать, роль, пара, появиться, данными, продуктовый, пора, бывать, число, способ, форма, искусственный, причина, данные, интеллект, многие, код, функция

 
Тема 4 :
страна, быстрый, новость, социальный, канал, знать, боль, открывать, конференция, дизайн, присоединиться, рекомендовать, европа, получение, що, ручной, продуктовый, собираться, подготовить, кожный

 
Тема 5 :
мысль, пройти, внимание, рабочий, время, первый, практика, почему, стать, видео, год, больший, жизнь, неделя, обучение, день, результат, момент, полезный, это

 
Тема 6 :
инвестор, отличный, совет, декабрь, инструмент